In [1]:
from urllib import *
import json
import regex as re
import PySimpleGUI as sg
import webbrowser
import sys
import pprint
from dataclasses import dataclass
import hyperlink
import requests
import time

In [2]:
# App configuration
_domain = "http://localhost:8983"
_url_comment = _domain+"/solr/comment"
_url_comment_select = _url_comment+"/select"
_url_comment_spell  = _url_comment+"/spell"

# defaults
_rows = 10
_params = {
    "q":"*",
    "wt":"json",
    "rows":_rows,
    "start":0,
    "indent":True,   
}


In [3]:
def doQuery(params=_params, spellcheck=False):
    # build query url
    url = _url_comment_select
    if spellcheck:
        url = _url_comment_spell
    response = requests.get(url, params=params)
    json = response.json()
    
    # display query result
    metadata = parseMetadata(json)
    showMetadata(metadata)
    if (spellcheck):
        showSpellCorrection(json)
    showDocs(json)
    return metadata, json
    
def parseMetadata(response):
    header1 = response["responseHeader"]
    header2 = response["response"]
    qTime = header1['QTime']
    q     = header1["params"]["q"]
    num_found = header2['numFound']
    start = header2['start']
    end   = start + len(header2['docs'])
    return qTime, num_found, start, end, q
    
def showMetadata(metadata):
    qTime, num_found, start, end, q = metadata
    print(f"Seach for {q}")
    if (num_found >0):
        print(f"found {num_found} results in {qTime} ms, showing {start+1} to {end}..")
    else:
        print(f'found {num_found} results in {qTime} ms')
    
def showSpellCorrection(response):
    if "spellcheck" not in response:
        return
    infoSpell = response['spellcheck']
    if (infoSpell['correctlySpelled']==False):
        # get original query terms
        qTerms = response["responseHeader"]["params"]["q"]
        suggestions = infoSpell['suggestions']
        if len(suggestions)==0:
            return
        # fix spell error
        for i in range(0, len(suggestions), 2):
            term = suggestions[i]
            infoSuggestion = suggestions[i+1]
            first = infoSuggestion['suggestion'][0]['word']
            qTerms = qTerms.replace(term, first)
        print(f"Do you mean `{qTerms}`?")
    
def showDocs(response):
    docs  = response['response']['docs']
    start = response["response"]['start']
    if len(docs) == 0:
        print("no data, try with other keywords")
    for i in range(len(docs)):
        index = start+i+1
        print("="*30, f'result {index}', "="*30)
        showDoc(docs[i])
    return docs
    
        
def showDoc(doc):
    print("Restaurant: ", doc["restaurant"])
    print("Type: ",       ",".join(doc["type"]))
    print("Reviewer: ",   doc["reviewer"][0])
    print("Rating: ",     doc['rating'])
    print("Comment: ",    doc['comment'])
    print("Semantic: ",   doc['pred_label'])
    print("Score: ",      doc['score'])



In [4]:
# params = _params.copy()
# params["q"] = "amasing food"

# doQuery(params=params, spellcheck=False)

In [ ]:
# GUI component
layout = [[sg.Combo(['Search', 'Spell check'], default_value='Search', key="_type_"), 
           sg.InputText(size=(30,10)), 
           sg.Button('Search', bind_return_key=True, key="_SEARCH_")],
          #[sg.Output(font = ["Arial", 16], size=(180,41), key = '_output_')],
          [sg.Output( size=(100,30), key = '_output_')],
          [sg.Button('Previous page', bind_return_key=True, key="_prev_"), 
           sg.Button('Next page', bind_return_key=True, key="_next_"),
           sg.Button("Go to first page", bind_return_key=True, key="_first_"),
           sg.Button('Go to last page', bind_return_key=True, key='_last_')]
         ]
window = sg.Window("Demo Search Engine", layout, resizable=True)

# Program code
typeSearch = ['_SEARCH_', '_next_', '_prev_', '_first_', '_last_']
total = 0
while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED:
        break
    if event in typeSearch:
        window['_output_'].update(value='')
        # decide start based on type
        if event =='_prev_':
            start = max(0, start-_rows)
        elif event =='_next_':
            start = min(max(0,total-_rows), start+_rows)
        elif event =='_last_':
            start = max(0, total-_rows)
        else:
            start = 0
        
        # build params
        param = _params.copy()
        param["q"] = str(values[0])
        param["start"] = start
        # query and display
        spellcheck = values['_type_']=='Spell check'
        metadata, json = doQuery(param, spellcheck)
        total = metadata[1]
window.close()